In [2]:
import pandas as pd
from nlpia.data.loaders import get_data
pd.options.display.width=120
sms=get_data('sms-spam')
index=['sms{}{}'.format(i,'!'*j) for (i,j) in zip(range(len(sms)),sms.spam)]
sms.index=index
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize
tfidf=TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs=tfidf.fit_transform(raw_documents=sms.text).toarray()

In [3]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(tfidf_docs,sms.spam.values,test_size=0.2,random_state=271828)


In [4]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda=LDA(n_components=1)

In [5]:
lda.fit(tfidf_docs,sms.spam)


LinearDiscriminantAnalysis(n_components=1)

In [6]:
sms['lda_spaminess']=lda.predict(tfidf_docs)

In [7]:
(sms.spam==sms.lda_spaminess).sum()

4837

In [8]:
len(sms)

4837

In [9]:
from sklearn.model_selection import cross_val_score
lda=LDA(n_components=1)
scores=cross_val_score(lda,tfidf_docs,sms.spam,cv=10)
"准确率:{:.2f}(+/-{:.2f})".format(scores.mean(),scores.std()*2)

'准确率:0.96(+/-0.02)'

In [10]:
lda=LDA(n_components=1)
lda.fit(x_train,y_train)
lda.score(x_test,y_test)

0.9628099173553719

In [11]:
lda=LDA(n_components=1)
lda.fit(x_train,y_train)

LinearDiscriminantAnalysis(n_components=1)

In [39]:
from sklearn.decomposition import LatentDirichletAllocation as LDiA
ldia=LDiA(n_components=16,learning_method='batch')

In [40]:
ldia=ldia.fit(tfidf_docs)

In [41]:
ldia_topic_vectors=ldia.transform(tfidf_docs)
columns32=['topic{}'.format(i) for i in range(ldia.n_components)]
ldia_topic_vectors=pd.DataFrame(ldia_topic_vectors,index=index,columns=columns32)

In [42]:
ldia_topic_vectors

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15
sms0,0.011198,0.011198,0.011198,0.011198,0.011198,0.356530,0.486695,0.011198,0.011198,0.011198,0.011198,0.011198,0.011198,0.011198,0.011198,0.011198
sms1,0.017648,0.017648,0.017648,0.017648,0.017648,0.017648,0.450083,0.017648,0.017648,0.017648,0.017648,0.302843,0.017648,0.017648,0.017648,0.017648
sms2!,0.009711,0.009711,0.009711,0.009711,0.009711,0.009711,0.009711,0.405771,0.009711,0.009711,0.009711,0.458278,0.009711,0.009711,0.009711,0.009711
sms3,0.122015,0.015563,0.015563,0.015563,0.015563,0.015563,0.660104,0.015563,0.015563,0.015563,0.015563,0.015563,0.015563,0.015563,0.015563,0.015563
sms4,0.014342,0.014342,0.014342,0.014342,0.014342,0.014342,0.014342,0.014342,0.014342,0.014342,0.472950,0.014342,0.326268,0.014342,0.014342,0.014342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sms4832!,0.009800,0.009800,0.009800,0.009800,0.009800,0.009800,0.009800,0.009800,0.009800,0.009800,0.009800,0.648584,0.009800,0.214221,0.009800,0.009800
sms4833,0.308414,0.016394,0.016394,0.016394,0.016394,0.016394,0.462073,0.016394,0.016394,0.016394,0.016394,0.016394,0.016394,0.016394,0.016394,0.016394
sms4834,0.013936,0.013936,0.013936,0.013936,0.013936,0.013936,0.013936,0.013936,0.013936,0.013936,0.322991,0.481907,0.013936,0.013936,0.013936,0.013936
sms4835,0.010823,0.010823,0.010823,0.010823,0.010823,0.010823,0.625231,0.010823,0.223250,0.010823,0.010823,0.010823,0.010823,0.010823,0.010823,0.010823


In [43]:
x_train,x_test,y_train,y_test=train_test_split(ldia_topic_vectors.values,sms.spam.values,test_size=0.2,random_state=271828)


In [44]:
lda=LDA(n_components=1,solver='svd',store_covariance=False,shrinkage=None,priors=None,tol=0.00001)
lda.fit(x_train,y_train)

LinearDiscriminantAnalysis(n_components=1, tol=1e-05)

In [45]:
lda.score(x_test,y_test)

0.8801652892561983

In [46]:
lda=LDA(n_components=1,solver='svd',store_covariance=False,shrinkage=None,priors=None,tol=0.00001)

In [47]:
scores=cross_val_score(lda,ldia_topic_vectors,sms.spam,cv=10)
"准确率:{:.2f}(+/-{:.2f})".format(scores.mean(),scores.std()*2)

'准确率:0.86(+/-0.02)'

In [48]:
from sklearn.decomposition import PCA
pca=PCA(n_components=16)
pca=pca.fit(tfidf_docs)
pca_topic_vectors=pca.transform(tfidf_docs)
colums=['topic{}'.format(i) for i in range(pca.n_components)]
pca_topic_vectors=pd.DataFrame(pca_topic_vectors,columns=colums,index=index)


In [49]:
pca_topic_vectors.head()

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15
sms0,0.201169,0.002722,0.037228,0.010963,-0.019496,-0.053075,0.039710,-0.065104,0.011216,-0.081477,0.011095,-0.014893,-0.003493,-0.022887,-0.018399,0.048606
sms1,0.404380,-0.093877,-0.077510,0.050900,0.099974,0.047044,0.022954,0.065189,0.022957,-0.023891,-0.003125,0.036879,0.039188,-0.011194,0.052755,-0.048924
sms2!,-0.030458,-0.048076,0.090197,-0.067028,0.090846,-0.043121,-0.000488,-0.001084,-0.055753,0.050219,0.124908,0.024711,0.025158,-0.044520,-0.023472,0.056638
sms3,0.329047,-0.032793,-0.034566,-0.015802,0.052092,0.055648,-0.165269,-0.074037,0.061994,-0.107182,0.024422,0.023438,0.061472,-0.034343,0.029585,-0.067398
sms4,0.002156,0.030817,0.038340,0.033888,-0.074634,-0.092604,-0.043691,0.061525,-0.045462,0.029576,0.029627,-0.007843,0.031439,0.005006,-0.091374,-0.010080


In [50]:
x_train,x_test,y_train,y_test=train_test_split(pca_topic_vectors.values,sms.spam.values,test_size=0.2,random_state=271828)

In [51]:
lda=LDA(n_components=1,solver='svd',store_covariance=False,shrinkage=None,priors=None,tol=0.00001)
lda.fit(x_train,y_train)

LinearDiscriminantAnalysis(n_components=1, tol=1e-05)

In [52]:
lda.score(x_test,y_test)

0.9617768595041323

In [53]:
lda=LDA(n_components=1,solver='svd',store_covariance=False,shrinkage=None,priors=None,tol=0.00001)
scores=cross_val_score(lda,pca_topic_vectors,sms.spam,cv=10)
"准确率:{:.2f}(+/-{:.2f})".format(scores.mean(),scores.std()*2)

'准确率:0.96(+/-0.02)'

In [56]:
from sklearn.decomposition import TruncatedSVD
svd=TruncatedSVD(n_components=16,n_iter=100)
svd_topic_vectors=svd.fit_transform(tfidf_docs)

In [57]:
svd_topic_vectors=pd.DataFrame(svd_topic_vectors,columns=colums,index=index)

In [58]:
svd_topic_vectors

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15
sms0,0.117168,0.197241,0.001291,0.046938,0.029898,0.023255,-0.055717,0.066620,-0.079180,-0.070192,-0.018184,-0.002791,0.007905,0.004756,-0.035671,-0.045036
sms1,0.111082,0.420921,-0.088945,0.001267,0.005467,-0.095879,0.051561,0.034003,0.046031,0.015792,-0.032491,-0.016574,-0.035084,0.043458,-0.022654,0.070904
sms2!,0.116148,-0.047167,-0.052929,0.018714,0.101805,-0.074445,-0.045453,0.009717,0.021123,-0.026668,0.140838,-0.041963,-0.024075,0.028729,-0.012834,-0.067845
sms3,0.136377,0.342812,-0.030319,-0.003396,0.120083,-0.010570,0.033826,-0.118974,-0.100092,-0.033333,-0.069072,-0.081043,-0.028591,0.078012,-0.036653,0.073147
sms4,0.117986,-0.001852,0.029035,0.048743,-0.027100,0.067796,-0.097089,-0.042936,0.079900,0.006323,0.020471,-0.044011,0.001253,0.030746,0.054195,-0.029297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sms4832!,0.305045,-0.098430,-0.163711,0.043858,0.278112,-0.115736,0.072404,-0.049639,-0.038576,-0.011360,-0.092641,0.063312,-0.015795,0.113507,-0.129220,0.117501
sms4833,0.122249,0.064666,0.077193,-0.034610,0.044142,0.041659,0.041438,-0.039057,-0.026100,-0.002505,0.211383,0.102310,-0.009561,-0.087287,-0.012005,-0.192491
sms4834,0.206329,0.099854,0.044925,0.041133,-0.005840,-0.014247,-0.010746,-0.020285,-0.002655,-0.092163,-0.057110,0.072065,0.097603,-0.005196,0.047922,0.016999
sms4835,0.168097,-0.023123,0.006503,-0.009782,-0.009180,0.061073,-0.105011,-0.028923,0.077440,-0.054957,0.073954,0.038780,-0.029359,-0.002064,-0.045830,0.045585


In [59]:
x_train,x_test,y_train,y_test=train_test_split(svd_topic_vectors.values,sms.spam.values,test_size=0.2,random_state=271828)

In [60]:
lda=LDA(n_components=1,solver='svd',store_covariance=False,shrinkage=None,priors=None,tol=0.00001)
lda.fit(x_train,y_train)
lda.score(x_test,y_test)

0.9638429752066116

In [62]:
lda=LDA(n_components=1,solver='svd',store_covariance=False,shrinkage=None,priors=None,tol=0.00001)
scores=cross_val_score(lda,svd_topic_vectors,sms.spam,cv=10)
"准确率:{:.2f}(+/-{:.2f})".format(scores.mean(),scores.std()*2)

'准确率:0.96(+/-0.02)'